In [1]:
import numpy as np
import os
import skimage
import utils as imgTrans
import fileIO as importFile
import matplotlib.pyplot as plt
import mlUtils as mL

In [2]:
def oneHotEncoding(speciesDic):
    curList = []
    for key in speciesDic:
        curList.append(speciesDic[key])
    speciesSet = set(curList)
    
    newDic = {}
    cnt = 0
    for key in speciesSet:
        newDic[key] = cnt
        cnt = cnt + 1
    return(cnt,newDic)

In [3]:
allFiles = importFile.readFileToDic()
numLabel,oneHotDic = oneHotEncoding(allFiles)

numLayers = 10
numRows = 32
numCols = 32

curPath = '../../data/oryzeae/'

tX = np.empty((numLayers*numCols*numRows))
tY = np.empty((1,numLabel))
vX = np.empty((numLayers*numCols*numRows))
vY = np.empty((1,numLabel))
testX = np.empty((numLayers*numCols*numRows))
testY = np.empty((1,numLabel))

for file in os.listdir(curPath):
    tiffFile = curPath+file
    species = allFiles[file]
    
    im = skimage.io.imread(tiffFile)
    im2 = skimage.transform.resize(im,(numLayers,numRows,numCols),mode='reflect')
    
    tmp = np.ndarray.flatten(im2)
    tX = np.vstack((tX,tmp))
    
    tmp = np.zeros((1,numLabel))
    curLabel = oneHotDic[species]
    tmp[0][curLabel] = 1
    tY = np.vstack((tY,tmp))
    
    allImg = imgTrans.manyRandImg(im2,20)
    for i in allImg:
        randNum = np.random.rand((1))
        tmp = np.ndarray.flatten(i)
        
        if randNum < .8:
            tX = np.vstack((tX,tmp))
            
            tmp = np.zeros((1,numLabel))
            curLabel = oneHotDic[species]
            tmp[0][curLabel] = 1
            tY = np.vstack((tY,tmp))
        elif randNum < .9:
            vX = np.vstack((vX,tmp))
            
            tmp = np.zeros((1,numLabel))
            curLabel = oneHotDic[species]
            tmp[0][curLabel] = 1
            vY = np.vstack((vY,tmp))
        else:
            testX = np.vstack((testX,tmp))
            
            tmp = np.zeros((1,numLabel))
            curLabel = oneHotDic[species]
            tmp[0][curLabel] = 1
            testY = np.vstack((testY,tmp))

In [4]:
tX = np.delete(tX, (0), axis=0)
tY = np.delete(tY, (0), axis=0)
vX = np.delete(vX, (0), axis=0)
vY = np.delete(vY, (0), axis=0)
testX = np.delete(testX, (0), axis=0)
testY = np.delete(testY, (0), axis=0)

In [5]:
wHat = mL.train(tX,tY,.1)

In [6]:
Y_hat = mL.predict(wHat, tX)
print("train: "+str(mL.accuracy(tY, Y_hat)))

Y_hat = mL.predict(wHat, vX)
print("validation: "+str(mL.accuracy(vY, Y_hat)))

Y_hat = mL.predict(wHat, testX)
print("test: "+str(mL.accuracy(testY, Y_hat)))

train: 0.999038461538
validation: 0.779661016949
test: 0.756097560976
